# Яппаров Айнур и Матниязов Азизбек, 4232М
# Вариант 7 - прочность бетона на сжатие
# Метод преобразования - Стандартизация
# Регрессионная модель - SVM
# Ансамблевая модель - Градиентный бустинг

Импорт необходимых библиотек

In [3]:
import numpy as np
from pandas import read_csv, DataFrame
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
%matplotlib inline

Загрузка датасета

In [4]:
df = read_csv(r'dataset_for_lab1_V7.csv', delimiter=(','), index_col=0)
df

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,NaN,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,NaN,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,NaN,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,NaN,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


# Формируем 4 набора данных 

### Исходный набор

In [5]:
original_data = df.copy()
mean = original_data['fly_ash'].mean()
original_data['fly_ash'].fillna(mean, inplace=True)
original_data

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.000000,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.000000,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,52.169556,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,52.169556,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.000000,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,52.169556,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,52.169556,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.600000,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.000000,175.6,11.3,989.6,788.9,28,32.77


### Преобразованный данные исходного набора

In [6]:
rescaled_data = original_data.copy()
scaler = StandardScaler().fit(rescaled_data)
rescaled_data = DataFrame(scaler.fit_transform(rescaled_data))
rescaled_data

,0,1,2,3,4,5,6,7,8
0,2.477915,-0.856888,-1.197426,-0.916764,-0.620448,0.863154,-1.217670,-0.279733,2.645408
1,2.477915,-0.856888,-1.197426,-0.916764,-0.620448,1.056164,-1.217670,-0.279733,1.561421
2,0.491425,0.795526,0.000000,2.175461,-1.039143,-0.526517,-2.240917,3.553066,0.266627
3,0.491425,0.795526,0.000000,2.175461,-1.039143,-0.526517,-2.240917,5.057677,0.313340
4,-0.790459,0.678408,-1.197426,0.488793,-1.039143,0.070527,0.647884,4.978487,0.507979
...,...,...,...,...,...,...,...,...,...
1025,-0.045645,0.488235,0.000000,-0.092171,0.451410,-1.323005,-0.065893,-0.279733,0.506781
1026,0.392819,-0.856888,0.000000,0.676200,0.702626,-1.994680,0.496893,-0.279733,-0.277762
1027,-1.270088,0.759579,1.295224,0.521589,-0.017528,-1.036064,0.080107,-0.279733,-0.725729
1028,-1.168610,1.308065,-1.197426,-0.279579,0.853356,0.214641,0.191166,-0.279733,-0.182539


### Формируем набор данных существенных признаков

In [7]:
original_inform = df.copy()
original_inform['water_cement_ratio'] = original_inform['water'] / original_inform['cement']
original_inform['superplasticizer_per_water'] = original_inform['superplasticizer'] / original_inform['water']
original_inform['slag_per_coarse_aggregate'] = original_inform['blast_furnace_slag'] / original_inform['coarse_aggregate']
col = original_inform.pop('concrete_compressive_strength')
original_inform.insert(len(original_inform.columns), col.name, col)
original_inform

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,water_cement_ratio,superplasticizer_per_water,slag_per_coarse_aggregate,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,0.300000,0.015432,0.000000,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,0.300000,0.015432,0.000000,61.89
2,332.5,142.5,NaN,228.0,0.0,932.0,594.0,270,0.685714,0.000000,0.152897,40.27
3,332.5,142.5,NaN,228.0,0.0,932.0,594.0,365,0.685714,0.000000,0.152897,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,0.966767,0.000000,0.135323,44.30
...,...,...,...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,NaN,179.6,8.9,870.1,768.3,28,0.649783,0.049555,0.133318,44.28
1026,322.2,0.0,NaN,196.0,10.4,817.9,813.4,28,0.608318,0.053061,0.000000,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,1.297643,0.031655,0.156208,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,1.103708,0.064351,0.188662,32.77


In [8]:
original_inform.drop(['fly_ash', 'age'], axis=1, inplace=True)
original_inform

,cement,blast_furnace_slag,water,superplasticizer,coarse_aggregate,fine_aggregate,water_cement_ratio,superplasticizer_per_water,slag_per_coarse_aggregate,concrete_compressive_strength
0,540.0,0.0,162.0,2.5,1040.0,676.0,0.300000,0.015432,0.000000,79.99
1,540.0,0.0,162.0,2.5,1055.0,676.0,0.300000,0.015432,0.000000,61.89
2,332.5,142.5,228.0,0.0,932.0,594.0,0.685714,0.000000,0.152897,40.27
3,332.5,142.5,228.0,0.0,932.0,594.0,0.685714,0.000000,0.152897,41.05
4,198.6,132.4,192.0,0.0,978.4,825.5,0.966767,0.000000,0.135323,44.30
...,...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,179.6,8.9,870.1,768.3,0.649783,0.049555,0.133318,44.28
1026,322.2,0.0,196.0,10.4,817.9,813.4,0.608318,0.053061,0.000000,31.18
1027,148.5,139.4,192.7,6.1,892.4,780.0,1.297643,0.031655,0.156208,23.70
1028,159.1,186.7,175.6,11.3,989.6,788.9,1.103708,0.064351,0.188662,32.77


### Формируем набор преобразованных данных существенных признаков

In [9]:
scaler = StandardScaler().fit(original_inform)
rescaled_inform = DataFrame(scaler.fit_transform(original_inform))
rescaled_inform

,0,1,2,3,4,5,6,7,8,9
0,2.477915,-0.856888,-0.916764,-0.620448,0.863154,-1.217670,-1.428268,-0.561907,-0.856648,2.645408
1,2.477915,-0.856888,-0.916764,-0.620448,1.056164,-1.217670,-1.428268,-0.561907,-0.856648,1.561421
2,0.491425,0.795526,2.175461,-1.039143,-0.526517,-2.240917,-0.199303,-0.956453,0.813492,0.266627
3,0.491425,0.795526,2.175461,-1.039143,-0.526517,-2.240917,-0.199303,-0.956453,0.813492,0.313340
4,-0.790459,0.678408,0.488793,-1.039143,0.070527,0.647884,0.696189,-0.956453,0.621526,0.507979
...,...,...,...,...,...,...,...,...,...,...
1025,-0.045645,0.488235,-0.092171,0.451410,-1.323005,-0.065893,-0.313788,0.310489,0.599625,0.506781
1026,0.392819,-0.856888,0.676200,0.702626,-1.994680,0.496893,-0.445904,0.400142,-0.856648,-0.277762
1027,-1.270088,0.759579,0.521589,-0.017528,-1.036064,0.080107,1.750427,-0.147131,0.849659,-0.725729
1028,-1.168610,1.308065,-0.279579,0.853356,0.214641,0.191166,1.132511,0.688778,1.204165,-0.182539


# Разделение каждого набора на обучающую и тестовую выборки

In [10]:
test_size = 0.2
seed = 5

# Исходный набор

original_data_X = original_data.iloc[:, :-1]
original_data_Y = original_data['concrete_compressive_strength']

original_data_X_train, original_data_X_test, original_data_Y_train, original_data_Y_test = \
train_test_split(original_data_X, original_data_Y, test_size=test_size, random_state=seed)

# Преобразованные данные исходного набора

rescaled_data_X = rescaled_data.iloc[:, :-1]
rescaled_data_Y = rescaled_data.iloc[:, -1]

rescaled_data_X_train, rescaled_data_X_test, rescaled_data_Y_train, rescaled_data_Y_test = \
train_test_split(rescaled_data_X, rescaled_data_Y, test_size=test_size, random_state=seed)

# Набор данных существенных признаков

original_inform_X = original_inform.iloc[:, :-1]
original_inform_Y = original_inform['concrete_compressive_strength']

original_inform_X_train, original_inform_X_test, original_inform_Y_train, original_inform_Y_test = \
train_test_split(original_inform_X, original_inform_Y, test_size=test_size, random_state=seed)

# Набор преобразованных данных существенных признаков

rescaled_inform_X = rescaled_inform.iloc[:, :-1]
rescaled_inform_Y = rescaled_inform.iloc[:, -1]

rescaled_inform_X_train, rescaled_inform_X_test, rescaled_inform_Y_train, rescaled_inform_Y_test = \
train_test_split(rescaled_inform_X, rescaled_inform_Y, test_size=test_size, random_state=seed)

# Формиурем датасет обучающих данных 

In [11]:
datasets = [{'X_train': original_data_X_train, 'Y_train': original_data_Y_train,
             'name': 'original_data'},
            {'X_train': rescaled_data_X_train, 'Y_train': rescaled_data_Y_train,
             'name': 'original_rescaled_data'},
            {'X_train': original_inform_X_train, 'Y_train': original_inform_Y_train,
             'name': 'inform_data'},
            {'X_train': rescaled_inform_X_train, 'Y_train': rescaled_inform_Y_train,
             'name': 'rescaled_inform_data'}]

# Реализация SVM

### Для начала на обучающей выборке определеим тип ядра, которое лучше всего подойдет к наборам данных, и набор данных, который даст лучшие оценки RMSE и R2

In [12]:
print('Линейное ядро\n')
for dataset in datasets:
    svr_model = SVR(kernel='linear')
    svr_model.fit(dataset.get('X_train'), dataset.get('Y_train'))
    prediction = svr_model.predict(dataset.get('X_train'))
    rmse = np.sqrt(mean_squared_error(dataset.get('Y_train'), prediction))
    r2 = r2_score(dataset.get('Y_train'), prediction)
    print(f'RMSE для обучающей выборки набора данных {dataset.get("name")}:', '%.4f' %rmse)
    print(f'R2 для обучающей выборки набора данных {dataset.get("name")}:', '%.4f' %r2)

print()
print('Полиномиальное ядро\n')
for dataset in datasets:
    svr_model = SVR(kernel='poly')
    svr_model.fit(dataset.get('X_train'), dataset.get('Y_train'))
    prediction = svr_model.predict(dataset.get('X_train'))
    rmse = np.sqrt(mean_squared_error(dataset.get('Y_train'), prediction))
    r2 = r2_score(dataset.get('Y_train'), prediction)
    print(f'RMSE для обучающей выборки набора данных {dataset.get("name")}:', '%.4f' %rmse)
    print(f'R2 для обучающей выборки набора данных {dataset.get("name")}:', '%.4f' %r2)

Линейное ядро

RMSE для обучающей выборки набора данных original_data: 10.8361
R2 для обучающей выборки набора данных original_data: 0.5949
RMSE для обучающей выборки набора данных original_rescaled_data: 0.6637
R2 для обучающей выборки набора данных original_rescaled_data: 0.5763
RMSE для обучающей выборки набора данных inform_data: 12.7684
R2 для обучающей выборки набора данных inform_data: 0.4376
RMSE для обучающей выборки набора данных rescaled_inform_data: 0.7600
R2 для обучающей выборки набора данных rescaled_inform_data: 0.4445

Полиномиальное ядро

RMSE для обучающей выборки набора данных original_data: 12.4345
R2 для обучающей выборки набора данных original_data: 0.4666
RMSE для обучающей выборки набора данных original_rescaled_data: 0.5592
R2 для обучающей выборки набора данных original_rescaled_data: 0.6993
RMSE для обучающей выборки набора данных inform_data: 14.0322
R2 для обучающей выборки набора данных inform_data: 0.3207
RMSE для обучающей выборки набора данных rescaled

### Для обоих типов ядер лучшим набором является исходный набор преобразованных данных. Полиномиальное ядро на этом наборе дает лучшие оценки RMSE и R2 - 0,56 и 0,7 соответственно

# Реализация градиентного бустинга

### Как и на предыдущем шаге, определим тип ядра и набор данных для градиентного бустинга, базовая модель - SVR

In [13]:
print('Линейное ядро\n')
for dataset in datasets:
    svr_model = SVR(kernel='linear')
    gbr_model = GradientBoostingRegressor(init=svr_model)
    gbr_model.fit(dataset.get('X_train'), dataset.get('Y_train'))
    prediction = gbr_model.predict(dataset.get('X_train'))
    rmse = np.sqrt(mean_squared_error(dataset.get('Y_train'), prediction))
    r2 = r2_score(dataset.get('Y_train'), prediction)
    print(f'RMSE для обучающей выборки набора данных {dataset.get("name")}:', '%.4f' %rmse)
    print(f'R2 для обучающей выборки набора данных {dataset.get("name")}:', '%.4f' %r2)

print()
print('Полиномиальное ядро\n')
for dataset in datasets:
    svr_model = SVR(kernel='poly')
    gbr_model = GradientBoostingRegressor(init=svr_model)
    gbr_model.fit(dataset.get('X_train'), dataset.get('Y_train'))
    prediction = gbr_model.predict(dataset.get('X_train'))
    rmse = np.sqrt(mean_squared_error(dataset.get('Y_train'), prediction))
    r2 = r2_score(dataset.get('Y_train'), prediction)
    print(f'RMSE для обучающей выборки набора данных {dataset.get("name")}:', '%.4f' %rmse)
    print(f'R2 для обучающей выборки набора данных {dataset.get("name")}:', '%.4f' %r2)

Линейное ядро

RMSE для обучающей выборки набора данных original_data: 4.0077
R2 для обучающей выборки набора данных original_data: 0.9446
RMSE для обучающей выборки набора данных original_rescaled_data: 0.2444
R2 для обучающей выборки набора данных original_rescaled_data: 0.9425
RMSE для обучающей выборки набора данных inform_data: 10.2731
R2 для обучающей выборки набора данных inform_data: 0.6359
RMSE для обучающей выборки набора данных rescaled_inform_data: 0.6186
R2 для обучающей выборки набора данных rescaled_inform_data: 0.6320

Полиномиальное ядро

RMSE для обучающей выборки набора данных original_data: 3.7512
R2 для обучающей выборки набора данных original_data: 0.9515
RMSE для обучающей выборки набора данных original_rescaled_data: 0.2768
R2 для обучающей выборки набора данных original_rescaled_data: 0.9263
RMSE для обучающей выборки набора данных inform_data: 10.2454
R2 для обучающей выборки набора данных inform_data: 0.6379
RMSE для обучающей выборки набора данных rescaled_i

### Для градиентного бустинга лучшей моделью является исходный набор данных, тип ядра - полиномиальное ядро с оценкой R2 = 0,95

# Поиск гиперпараметров для SVR

### Зададим диапазон значений для поиска по сетке, ядро - полиномиальное

In [14]:
params_svr = {
    'kernel': ['poly'],
    'C': [i for i in np.linspace(5, 20, num=10)],
    'degree': [i for i in np.arange(1, 11)],
    'coef0': [0.1, 0.2]
}

### Запустим поиск по сетке для определения лучших гиперпараметров на обучающей и тестовой выборках лучшей модели - исходном наборе преобразованных данных

In [15]:
model = SVR()
grid_search = GridSearchCV(model,
                           params_svr,
                           n_jobs=-1,
                           cv=2)
grid_search.fit(rescaled_data_X_train, rescaled_data_Y_train)
best_svr = grid_search.best_params_
print(f'Лучшие гиперпараметры для исходного набора преобразованных данных:', best_svr, '\n')

svr_model = SVR(**best_svr)
svr_model.fit(rescaled_data_X_train, rescaled_data_Y_train)
prediction_train = svr_model.predict(rescaled_data_X_train)
rmse_train = np.sqrt(mean_squared_error(rescaled_data_Y_train, prediction_train))
r2_train = r2_score(rescaled_data_Y_train, prediction_train)
print(f'RMSE для обучающей выборки:', '%.4f' %rmse_train)
print(f'R2 для обучающей выборки:', '%.4f' %r2_train, '\n')

prediction_test = svr_model.predict(rescaled_data_X_test)
rmse_test = np.sqrt(mean_squared_error(rescaled_data_Y_test, prediction_test))
r2_test = r2_score(rescaled_data_Y_test, prediction_test)
print(f'RMSE для тестовой выборки:', '%.4f' %rmse_test)
print(f'R2 для тестовой выборки:', '%.4f' %r2_test)

Лучшие гиперпараметры для исходного набора преобразованных данных: {'C': 11.666666666666668, 'coef0': 0.2, 'degree': 3, 'kernel': 'poly'} 

RMSE для обучающей выборки: 0.3149
R2 для обучающей выборки: 0.9046 

RMSE для тестовой выборки: 0.4406
R2 для тестовой выборки: 0.7693


### Как видно из расчетов, после определения гиперпараметров значения RMSE и R2 значительно улучшились

# Поиск гиперпараметров для градиентного бустинга с базовой моделью SVR

### Зададим диапазон значений для поиска по сетке 

In [16]:
params_gbr = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [50, 100, 200],
    'max_depth': [1, 2, 3, 4, 5]
}

In [17]:
base_model = SVR(kernel='poly', C=10, degree=10, coef0=0.2)
gradboost_model = GradientBoostingRegressor(init=base_model)
grid_search = GridSearchCV(gradboost_model,
                           params_gbr,
                           n_jobs=-1,
                           cv=2,)
grid_search.fit(original_data_X_train,
                original_data_Y_train)
best_gbr = grid_search.best_params_
print(f'Лучшие гиперпараметры для исходного набора преобразованных данных:', best_gbr, '\n')

svr_base = SVR(kernel='poly', C=10)
gbr_model = GradientBoostingRegressor(init=svr_base, **best_gbr)
gbr_model.fit(original_data_X_train, original_data_Y_train)
prediction_train = gbr_model.predict(original_data_X_train)
rmse_train = np.sqrt(mean_squared_error(original_data_Y_train, prediction_train))
r2_train = r2_score(original_data_Y_train, prediction_train)
print(f'RMSE для обучающей выборки:', '%.4f' %rmse_train)
print(f'R2 для обучающей выборки:', '%.4f' %r2_train, '\n')

prediction_test = gbr_model.predict(original_data_X_test)
rmse_test = np.sqrt(mean_squared_error(original_data_Y_test, prediction_test))
r2_test = r2_score(original_data_Y_test, prediction_test)
print(f'RMSE для тестовой:', '%.4f' %rmse_test)
print(f'R2 для тестовой:', '%.4f' %r2_test)

Лучшие гиперпараметры для исходного набора преобразованных данных: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200} 

RMSE для обучающей выборки: 2.0431
R2 для обучающей выборки: 0.9856 

RMSE для тестовой: 4.3835
R2 для тестовой: 0.9181


В первой лабораторной работе лучшей предиктивной моделью была полиномиальная регрессия 3 степени, на исходном преобразованном наборе данных с оценкой R2 = 0,82 на тестовых данных.

Во второй лабораторной работе лучшей моделью является градиентный бустинг на исходном наборе с R2 = 0,92 на тестовых данных, что выше, чем у полиномиальной регрессии.